# Imports
---

In [57]:
import random, inspect
from collections import Counter

# Problem Definition
---

### Constraints

In [58]:
# Minimum of classes to each class
REQUIREMENTS = {
    'M': 5, # Matemática 
    'T': 1, # Tecnologia (Mind Makers) 
    'F': 1, # Ed. Financeira 
    'L': 7, # Língua Portuguesa 
    'H': 2, # História 
    'G': 2, # Geografia 
    'R': 1, # Ensino Religioso 
    'A': 2, # Artes 
    'C': 2, # Ciências 
    'I': 5, # Inglês 
    'U': 1, # Música 
    'E': 1, # Educação Física 
}

In [59]:
# Number of groups of students
CLASSROOMS = 4

In [60]:
# Professors that can teach more than one subject
SIM_PROFESSORS = [
    ('M', 'T', 'F'), # Matemática, Tecnologia e Ed. Financeira
    ('H', 'G'), # História e Geografia
    ('A', 'U') # Artes e Música
]

### Solution example

In [61]:
case: str = ''.join([random.choice(list(REQUIREMENTS.keys())) for _ in range(CLASSROOMS * sum(REQUIREMENTS.values()))])
case

'ATFHIRCUCLEETEREGFHRCUUFCEFMHLCAEACCULLCEGUIFLHMFHGAMLALGMEUCATMFFHIULGIMAMATRCMUFTRUEURFEATTFRUIEHIMUEFFTIGEMRLMHUCFFGE'

# Utility functions
---

In [62]:
def separar_turmas(case: str, n: int = CLASSROOMS) -> list[str]:
    """ Separa o horário em turmas
    
    Args:
        case (str): Horário das aulas
        n (int, optional): Quantidade de turmas.
    
    Returns:
        list[str]: Lista de horários separados por turma
    """
    tamanho_grupo = len(case) // n
    turmas = []
    for i in range(n):
        inicio = i * tamanho_grupo
        fim = inicio + tamanho_grupo
        turmas.append(case[inicio:fim])
        
    return turmas
        
separar_turmas(case)

['ATFHIRCUCLEETEREGFHRCUUFCEFMHL',
 'CAEACCULLCEGUIFLHMFHGAMLALGMEU',
 'CATMFFHIULGIMAMATRCMUFTRUEURFE',
 'ATTFRUIEHIMUEFFTIGEMRLMHUCFFGE']

In [63]:
def separar_dias(turma: str) -> list[list[str]]:
    """ Separa a turma em dias
    
    Args:
        turma (str): Horário de uma turma
    
    Returns:
        list[list[str]]: Lista de horários separados por dia
    """
    dias: list[list[str]] = [['' for _ in range(6)] for _ in range(5)]
    
    for i, aula in enumerate(turma):
        dias[i // 6][i % 6] = aula
    
    return [''.join(dia) for dia in dias]

separar_dias(separar_turmas(case)[0])

['ATFHIR', 'CUCLEE', 'TEREGF', 'HRCUUF', 'CEFMHL']

In [64]:
def sintetizar_professores(case: str, sp: list[tuple[str]] = SIM_PROFESSORS) -> str:
    for p in sp:
        for c in p:
            case = case.replace(c, p[0])
    return case

sintetizar_professores(case)

'AMMHIRCACLEEMEREHMHRCAAMCEMMHLCAEACCALLCEHAIMLHMMHHAMLALHMEACAMMMMHIALHIMAMAMRCMAMMRAEARMEAMMMRAIEHIMAEMMMIHEMRLMHACMMHE'

# Avaliação

In [65]:
def quantidade_aulas(case: str, n: int, r: dict = REQUIREMENTS) -> int:
    """ Verifica se o horário tem a quantidade de aulas necessárias 
    
    Args:
        case (str): Horário das aulas
        r (dict): Requisitos de aulas por matéria
    
    Returns:
        int: Quantidade de aulas faltantes ou excedentes
    """
    
    # Conta a quantidade de aulas de cada matéria no horário especificado
    c: Counter = Counter(case)
    # e.g. {'M': 28, 'L': 28, 'H': 16, 'C': 8, 'R': 4, 'I': 20, 'F': 4}

    nota_quantidade: int =  sum( # Somatório
        abs( # Valor absoluto para obter a diferença entre a quantidade de aulas esperada e a quantidade de aulas no horário
            (r[materia]*n) - c.get(materia, 0) # Quantidade esperada daquela matéria - Quantidade de aulas daquela matéria no horário
        ) \
            for materia in r # Aplica a operação para cada matéria, e soma o resultado
    )
    
    return nota_quantidade

quantidade_aulas(separar_turmas(case)[0], n=1)

30

In [66]:
def bonus_aula_dupla(hdia: str | list[str]) -> float:
    """ Verifica se há aulas duplas
    
    Args:
        hdia (str | list[str]): Horário de um dia
    
    Returns:
        float: Bônus por aula dupla
    """
    bonus: float = 0.00
    ultima: str = ''
    i: int = 0
    
    for aula in hdia:
        if aula == ultima:
            if i <= 1:
                bonus += 0.01
            else:
                bonus -= 1.01
                
            i+=1
            
        else:
            i = 0
            
        ultima = aula
        
    return round(bonus, 2)

bonus_aula_dupla('AAABAA')

0.03

In [67]:
def validar_restricoes(cases: list[str]) -> int:
    """ Verifica se o horário atende as restrições"""
    # TODO:  Somar a distância entre o horário esperado x horário anotado pois incentiva mais a aproximação do horário esperado
    
    nota_restricoes: int = 0
    for turma in cases:
        nota_restricoes+= quantidade_aulas(turma, n=1)
        for d, dia in enumerate(separar_dias(turma)):
            if any (dia.count(aula) > 3 for aula in dia):
                nota_restricoes += 1
            nota_restricoes -= bonus_aula_dupla(dia)
            
            # Verifica se o horário da aula de inglês é após as 10:50
            if 'I' in dia and dia.find('I', 4) >= 4:
                nota_restricoes += dia.find('I', 4)
            
            # Verifica se tem aula de inglês
            if 'I' not in dia:
                nota_restricoes += 1
            
            # Verifica se o horário de História é após as 10:50
            if 'H' in dia and dia.find('H', 4) >= 4:
                nota_restricoes += dia.find('H', 4)
                
            # Verifica se o horário de Geografia é após as 10:50
            if 'G' in dia and dia.find('G', 4) >= 4:
                nota_restricoes += dia.find('G', 4)
                
            # Verifica se o horário de Educação Física é antes das 9:10
            if 'E' in dia and dia.find('E', 0, 2) <= 1 and dia.find('E', 0, 2) != -1:
                nota_restricoes += 1
                
            # Verifica se o dia da educação física não é sexta-feira
            if 'E' in dia and d != 4:
                nota_restricoes += 1
                
            # Verifica se o dia de Artes é segunda-feira, terça-feira ou sexta-feira
            if 'A' in dia and d not in [0, 1, 4]:
                nota_restricoes += sum(1 for aula in dia if aula == 'A')
            
            # Verifica se o dia de Música é segunda-feira, terça-feira ou sexta-feira
            if 'U' in dia and d not in [0, 1, 4]:
                nota_restricoes += sum(1 for aula in dia if aula == 'U')

    return nota_restricoes

validar_restricoes(separar_turmas(case))

180.92000000000002

In [68]:
def validar_sobreposicoes(strings: list[str]) -> int:    
    chars = set(list(''.join(strings)))
    positions = {char: {num: {} for num in range(5)} for char in chars}
    nota_sobreposicoes: int = 0
    
    for turma in strings:
        for j, dia in enumerate(separar_dias(turma)):
            for k, aula in enumerate(dia):
                if positions[aula][j] and k not in positions[aula][j]:
                    positions[aula][j].add(k)
                elif positions[aula][j] and k in positions[aula][j]:
                    nota_sobreposicoes += 1
                else:
                    positions[aula][j] = {k}
    
    return nota_sobreposicoes


validar_sobreposicoes(separar_turmas(case))

15

In [69]:
def fitting(case: str) -> int:
    """ Função de fitness"""
    cases: list[str] = separar_turmas(case)
    return round((validar_restricoes(cases) + validar_sobreposicoes([sintetizar_professores(t) for t in cases])), 2)

fitting(case)

206.92

# Export

In [70]:
raw: str = f'{globals()['_ih'][1]}{'\n'*3}'\
        + f'REQUIREMENTS = {REQUIREMENTS}{'\n'*3}'\
        + f'CLASSROOMS = {CLASSROOMS}{'\n'*3}'\
        + f'SIM_PROFESSORS = {SIM_PROFESSORS}{'\n'*3}'\
        + f'{'\n'*2}'.join([inspect.getsource(f) for f in [separar_turmas, separar_dias, sintetizar_professores, quantidade_aulas, bonus_aula_dupla, validar_restricoes, validar_sobreposicoes, fitting]])

In [71]:
with open('fitting_function.py', 'w', encoding='utf-8') as f:
    f.write(raw)

# Test

In [72]:
validar_restricoes(
    separar_turmas('LGILMCIMMMALHLILHMLGILCRUTFIEAILMUCTGLAICGMMLILRHILMHLMILEAFAILTLLMHIULALIGMCCIRHGMFLLIMMEGTGIAULILLMMIHMHMLRFMILCICEALL')
)

12.900000000000002

In [73]:
separar_turmas(
    'LGILMCIMMMALHLILHMLGILCRUTFIEAILMUCTGLAICGMMLILRHILMHLMILEAFAILTLLMHIULALIGMCCIRHGMFLLIMMEGTGIAULILLMMIHMHMLRFMILCICEALL'
)

['LGILMCIMMMALHLILHMLGILCRUTFIEA',
 'ILMUCTGLAICGMMLILRHILMHLMILEAF',
 'AILTLLMHIULALIGMCCIRHGMFLLIMME',
 'GTGIAULILLMMIHMHMLRFMILCICEALL']

In [74]:
validar_sobreposicoes(
    separar_turmas('LGILMCIMMMALHLILHMLGILCRUTFIEAILMUCTGLAICGMMLILRHILMHLMILEAFAILTLLMHIULALIGMCCIRHGMFLLIMMEGTGIAULILLMMIHMHMLRFMILCICEALL')
)

0

In [75]:
fitting(
    'FIULLLMLLICCMILLTRGGIMMMHHEIAAHHIUMMGGIMMMLLIFRTILLLCCAAIELLLLLIAAUIGGLLHMMICCFTHIRLLIMMMEIMMMCCIMMUAAIHGGLLLITHLRILLLEF'
)

-0.39

# Base benchmarking

In [76]:
import cProfile

In [77]:
def fit_multiple_elements():
    for _ in range(100):
        fitting('RLHMAAELLEGRATRACMLHTTRUILHILACAIGATIEMMMTAHLLICCTTRHUHALMTGHAGAIALCAAHELCGUHFHMRMURAFTRCGRACIEUIUITFHFEMEFEHIILGHLRIFRM')

cProfile.run('fit_multiple_elements()')

         71987 function calls in 0.052 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      100    0.005    0.000    0.025    0.000 1034947207.py:1(validar_restricoes)
      500    0.000    0.000    0.000    0.000 1034947207.py:39(<genexpr>)
      800    0.000    0.000    0.000    0.000 1034947207.py:43(<genexpr>)
    14000    0.003    0.000    0.005    0.000 1034947207.py:9(<genexpr>)
      100    0.017    0.000    0.057    0.001 1199850717.py:1(fitting)
      100    0.000    0.000    0.000    0.000 123720838.py:1(separar_turmas)
        1    0.000    0.000    0.036    0.036 1277134286.py:1(fit_multiple_elements)
      400    0.001    0.000    0.001    0.000 1909215109.py:1(sintetizar_professores)
      100    0.005    0.000    0.008    0.000 2307560999.py:1(validar_sobreposicoes)
     2000    0.001    0.000    0.002    0.000 268117936.py:1(bonus_aula_dupla)
      400    0.001    0.000    0.007    0.000 636192225.py:1(qua

In [78]:
cProfile.run('fitting(\"RLHMAAELLEGRATRACMLHTTRUILHILACAIGATIEMMMTAHLLICCTTRHUHALMTGHAGAIALCAAHELCGUHFHMRMURAFTRCGRACIEUIUITFHFEMEFEHIILGHLRIFRM\")')

         722 function calls in 0.001 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 1034947207.py:1(validar_restricoes)
        5    0.000    0.000    0.000    0.000 1034947207.py:39(<genexpr>)
        8    0.000    0.000    0.000    0.000 1034947207.py:43(<genexpr>)
      140    0.000    0.000    0.000    0.000 1034947207.py:9(<genexpr>)
        1    0.000    0.000    0.001    0.001 1199850717.py:1(fitting)
        1    0.000    0.000    0.000    0.000 123720838.py:1(separar_turmas)
        4    0.000    0.000    0.000    0.000 1909215109.py:1(sintetizar_professores)
        1    0.000    0.000    0.000    0.000 2307560999.py:1(validar_sobreposicoes)
       20    0.000    0.000    0.000    0.000 268117936.py:1(bonus_aula_dupla)
        4    0.000    0.000    0.000    0.000 636192225.py:1(quantidade_aulas)
       52    0.000    0.000    0.000    0.000 636192225.py:16(<genexpr>)